<a href="https://colab.research.google.com/github/Abin-j-g7/skillcraft_ML_03/blob/main/implementing_of_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"abinj10","key":"f9b644d7d7a5f09adab28c66039567b3"}'}

In [2]:
import os
os.makedirs("/root/.kaggle", exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json


In [3]:
!pip install kaggle


In [4]:
!kaggle competitions download -c dogs-vs-cats


 95% 775M/812M [00:03<00:00, 157MB/s]
100% 812M/812M [00:05<00:00, 170MB/s]


In [5]:
!kaggle datasets download -d <Kaggle · Playground Prediction Competition>/<Dogs vs. Cats>


/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `kaggle datasets download -d <Kaggle · Playground Prediction Competition>/<Dogs vs. Cats>'


In [6]:
!unzip dogs-vs-cats.zip -d dataset


Archive:  dogs-vs-cats.zip
  inflating: dataset/sampleSubmission.csv  
  inflating: dataset/test1.zip       
  inflating: dataset/train.zip       


In [7]:
import os
print(os.listdir("dataset"))


['train.zip', 'sampleSubmission.csv', 'test1.zip']


In [8]:
import zipfile
import os

# Unzip train.zip
with zipfile.ZipFile("dataset/train.zip", "r") as zip_ref:
    zip_ref.extractall("dataset/train")

# Unzip test1.zip
with zipfile.ZipFile("dataset/test1.zip", "r") as zip_ref:
    zip_ref.extractall("dataset/test")

print("Train files:", len(os.listdir("dataset/train")))
print("Test files:", len(os.listdir("dataset/test")))


Train files: 1
Test files: 1


In [12]:
import os, shutil, random

# Paths after unzipping
original_train_dir = "dataset/train/train"   # contains 25,000 cat/dog images
original_test_dir  = "dataset/test/test1"    # contains 12,500 test images

# Small dataset structure
base_dir = "dataset/cats_and_dogs_small"
train_dir = os.path.join(base_dir, "train")
val_dir   = os.path.join(base_dir, "validation")
test_dir  = os.path.join(base_dir, "test")

# Create directories
for d in [train_dir, val_dir, test_dir]:
    for cls in ["cats", "dogs"]:
        os.makedirs(os.path.join(d, cls), exist_ok=True)

# Get all train filenames
filenames = os.listdir(original_train_dir)

cats = [f for f in filenames if f.startswith("cat")]
dogs = [f for f in filenames if f.startswith("dog")]

random.shuffle(cats)
random.shuffle(dogs)

# Split: 10,000 train / 2,500 validation each
train_cats = cats[:10000]
val_cats   = cats[10000:12500]

train_dogs = dogs[:10000]
val_dogs   = dogs[10000:12500]

# Copy training cats
for fname in train_cats:
    shutil.copy(os.path.join(original_train_dir, fname),
                os.path.join(train_dir, "cats", fname))

# Copy validation cats
for fname in val_cats:
    shutil.copy(os.path.join(original_train_dir, fname),
                os.path.join(val_dir, "cats", fname))

# Copy training dogs
for fname in train_dogs:
    shutil.copy(os.path.join(original_train_dir, fname),
                os.path.join(train_dir, "dogs", fname))

# Copy validation dogs
for fname in val_dogs:
    shutil.copy(os.path.join(original_train_dir, fname),
                os.path.join(val_dir, "dogs", fname))

# Prepare test set (using 2500 each from test1 folder)
test_filenames = os.listdir(original_test_dir)
test_cats = test_filenames[:2500]
test_dogs = test_filenames[2500:5000]

for fname in test_cats:
    shutil.copy(os.path.join(original_test_dir, fname),
                os.path.join(test_dir, "cats", fname))

for fname in test_dogs:
    shutil.copy(os.path.join(original_test_dir, fname),
                os.path.join(test_dir, "dogs", fname))

# Verify counts
print("Training cats:", len(os.listdir(os.path.join(train_dir, "cats"))))
print("Validation cats:", len(os.listdir(os.path.join(val_dir, "cats"))))
print("Test cats:", len(os.listdir(os.path.join(test_dir, "cats"))))
print("Training dogs:", len(os.listdir(os.path.join(train_dir, "dogs"))))
print("Validation dogs:", len(os.listdir(os.path.join(val_dir, "dogs"))))
print("Test dogs:", len(os.listdir(os.path.join(test_dir, "dogs"))))


Training cats: 10000
Validation cats: 2500
Test cats: 2500
Training dogs: 10000
Validation dogs: 2500
Test dogs: 2500


In [1]:
import os
import numpy as np
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# -----------------------
# 1. Dataset Paths
# -----------------------
train_cat_dir = "dataset/cats_and_dogs_small/train/cats"
train_dog_dir = "dataset/cats_and_dogs_small/train/dogs"
val_cat_dir   = "dataset/cats_and_dogs_small/validation/cats"
val_dog_dir   = "dataset/cats_and_dogs_small/validation/dogs"
test_cat_dir  = "dataset/cats_and_dogs_small/test/cats"
test_dog_dir  = "dataset/cats_and_dogs_small/test/dogs"

# -----------------------
# 2. Helper to Load Images
# -----------------------
def load_images_from_folder(folder, label, target_size=(64,64)):
    images, labels = [], []
    for filename in tqdm(os.listdir(folder), desc=f"Loading {os.path.basename(folder)}"):
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=target_size)
        img_array = img_to_array(img) / 255.0
        images.append(img_array.flatten())
        labels.append(label)
    return np.array(images), np.array(labels)

# -----------------------
# 3. Load Train/Val/Test
# -----------------------
X_train_cats, y_train_cats = load_images_from_folder(train_cat_dir, 0)
X_train_dogs, y_train_dogs = load_images_from_folder(train_dog_dir, 1)
X_val_cats, y_val_cats     = load_images_from_folder(val_cat_dir, 0)
X_val_dogs, y_val_dogs     = load_images_from_folder(val_dog_dir, 1)
X_test_cats, y_test_cats   = load_images_from_folder(test_cat_dir, 0)
X_test_dogs, y_test_dogs   = load_images_from_folder(test_dog_dir, 1)

X_train = np.vstack((X_train_cats, X_train_dogs))
y_train = np.concatenate((y_train_cats, y_train_dogs))
X_val   = np.vstack((X_val_cats, X_val_dogs))
y_val   = np.concatenate((y_val_cats, y_val_dogs))
X_test  = np.vstack((X_test_cats, X_test_dogs))
y_test  = np.concatenate((y_test_cats, y_test_dogs))

print("Shapes:")
print("Train:", X_train.shape, y_train.shape)
print("Validation:", X_val.shape, y_val.shape)
print("Test:", X_test.shape, y_test.shape)

# -----------------------
# 4. PCA for Dim Reduction
# -----------------------
pca = PCA(n_components=300, random_state=42)
X_train_pca = pca.fit_transform(X_train)
X_val_pca   = pca.transform(X_val)
X_test_pca  = pca.transform(X_test)

print("After PCA:")
print("Train:", X_train_pca.shape)
print("Validation:", X_val_pca.shape)
print("Test:", X_test_pca.shape)

# -----------------------
# 5. Train SVM
# -----------------------
print("\nTraining SVM...")
svm = SVC(kernel='rbf', C=10, gamma=0.01)
svm.fit(X_train_pca, y_train)

# -----------------------
# 6. Evaluate
# -----------------------
y_val_pred = svm.predict(X_val_pca)
y_test_pred = svm.predict(X_test_pca)

print("\nValidation Accuracy:", accuracy_score(y_val, y_val_pred))
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))

print("\nClassification Report (Test):")
print(classification_report(y_test, y_test_pred, target_names=["Cat", "Dog"]))

print("\nConfusion Matrix (Test):")
print(confusion_matrix(y_test, y_test_pred))


Loading dogs: 100%|██████████| 2500/2500 [00:02<00:00, 850.00it/s]


Shapes:
Train: (20000, 12288) (20000,)
Validation: (5000, 12288) (5000,)
Test: (5000, 12288) (5000,)
After PCA:
Train: (20000, 300)
Validation: (5000, 300)
Test: (5000, 300)

Training SVM...

Validation Accuracy: 0.6254
Test Accuracy: 0.5

Classification Report (Test):
              precision    recall  f1-score   support

         Cat       0.50      0.66      0.57      2500
         Dog       0.50      0.34      0.40      2500

    accuracy                           0.50      5000
   macro avg       0.50      0.50      0.49      5000
weighted avg       0.50      0.50      0.49      5000


Confusion Matrix (Test):
[[1655  845]
 [1655  845]]
